In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db
from sqlalchemy import create_engine

In [2]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

In [ ]:
# Read the following data set:

In [81]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/"

dataset = "adult.data"
link = url + dataset
data = pd.read_csv(link, header= None)
adult_data=pd.DataFrame(data)
# adult.head(3)
sqladb=adult_data.rename(columns={0:"age",1:"workclass",2:"fnlwgt",3:"education",4:"education-num",5:"marital-status",6:"occupation",7:"relationship",8:"race",9:"sex",10:"capital-gain",11:"capital-loss",12:"hours-per-week",13:"native-country",14:"class"})

sqladb.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
# 1. Select 10 records from the adult sqladb

In [82]:
q= """SELECT 
    *
    FROM
        sqladb
    LIMIT 10;"""
pysqldf(q)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [ ]:
# 2. Show me the average hours per week of all men who are working in private sector

In [83]:
#Trim all string(Object) columns for blank spaces:

def trimAllColumns(df):
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)
sqladb = trimAllColumns(sqladb)

# Replace column name having “-“ with “_”

sqladb.columns = sqladb.columns.str.replace('-', '_')

sqladb.head(3)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [84]:
q="SELECT sex,workclass,avg(hours_per_week) from sqladb where sex='Male' and workclass='Private' ;"
pysqldf(q)

,sex,workclass,avg(hours_per_week)
0,Male,Private,42.221226


In [ ]:
# 3. Show me the frequency table for education, occupation and relationship, separately


In [85]:
pysqldf("SELECT education, count() from sqladb GROUP BY education")

,education,count()
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5355


In [53]:
pysqldf("SELECT occupation, count() from sqladb GROUP BY occupation")

,occupation,count()
0,?,1843
1,Adm-clerical,3770
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [54]:
pysqldf("SELECT relationship, count() from sqladb GROUP BY relationship")

,relationship,count()
0,Husband,13193
1,Not-in-family,8305
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


In [ ]:
# 4. Are there any people who are married, working in private sector and having a masters degree

In [86]:
z=pysqldf("SELECT marital_status,workclass,education from sqladb where  workclass='Private'and education='Masters' and marital_status like 'Married%';")
z.info()
print(z)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 3 columns):
marital_status    540 non-null object
workclass         540 non-null object
education         540 non-null object
dtypes: object(3)
memory usage: 12.7+ KB
            marital_status workclass education
0       Married-civ-spouse   Private   Masters
1       Married-civ-spouse   Private   Masters
2       Married-civ-spouse   Private   Masters
3       Married-civ-spouse   Private   Masters
4       Married-civ-spouse   Private   Masters
5       Married-civ-spouse   Private   Masters
6       Married-civ-spouse   Private   Masters
7       Married-civ-spouse   Private   Masters
8       Married-civ-spouse   Private   Masters
9       Married-civ-spouse   Private   Masters
10      Married-civ-spouse   Private   Masters
11      Married-civ-spouse   Private   Masters
12      Married-civ-spouse   Private   Masters
13      Married-civ-spouse   Private   Masters
14      Married-civ-spouse   Private

In [ ]:
# 5. What is the average, minimum and maximum age group for people working in different sector

In [87]:
pysqldf("SELECT workclass,avg(age),min(age),max(age) FROM sqladb GROUP BY  workclass;")

,workclass,avg(age),min(age),max(age)
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [ ]:
# 6. Calculate age distribution by country

In [88]:
pysqldf(" SELECT native_country,SUM(CASE WHEN age BETWEEN 1 AND 18 THEN 1 ELSE 0 END) AS [Under 18],SUM(CASE WHEN age BETWEEN 18 AND 50 THEN 1 ELSE 0 END) AS [18-50], SUM(CASE WHEN age BETWEEN 51 AND 120 THEN 1 ELSE 0 END) AS [51-120] FROM sqladb GROUP BY native_country;")

,native_country,Under 18,18-50,51-120
0,?,10,482,99
1,Cambodia,1,17,2
2,Canada,3,81,38
3,China,0,55,20
4,Columbia,1,46,13
5,Cuba,0,59,36
6,Dominican-Republic,1,57,13
7,Ecuador,0,26,2
8,El-Salvador,2,92,12
9,England,1,68,21


In [ ]:
# 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss

In [ ]:
pysqldf("ALTER TABLE sqladb ADD net_capital_gain AS (capital_gain-capital_loss) ;")

In [110]:
# One way of doing this.
sqladb["net_capital_gain"]=sqladb["capital_gain"]-sqladb["capital_loss"]
sqladb.head(2)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,net_capital_gain
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,2174
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
